In [1]:
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from statistics import mean
from datetime import date

In [2]:
# start_date = '2020-01-01'
# start_date = '2021-01-01' # start of previous cycle
# start_date = '2022-01-01' # start of downtrend
start_date = '2023-01-01' # start of uptrend
# start_date = '2023-04-10' # start of MAGS history
# start_date = '2024-02-28' # start of MAGX history

weeks = 52
end_date = date.today()
max_days = (end_date - date.fromisoformat(start_date)).days
min_history = max_days*0.8/7
rows = []
for symbol in [
    'SPY','QQQ','IGM','XLKS.MI','SMH','USD','TQQQ','FNGU','FNGS','MAGS',
    'MAGX','MAG7.L'
]:
    tickerData = yf.Ticker(symbol)
    history = tickerData.history(interval='1wk', start=start_date, end=date.today()) # prices are adjusted for splits
    changes = history.Close.pct_change(periods=weeks) if len(history) > min_history else pd.Series([])
    rows.append({
        'symbol': symbol,
        'weeks': len(history),
        'change': changes.mean(),
        'std': changes.std(),
        'score': changes.mean() / changes.std(),
    })

df = pd.DataFrame(rows).sort_values(by='score', ascending=False)
display(df)
df.to_csv('data/stocks-52w.csv', index=False)


,symbol,weeks,change,std,score
3,XLKS.MI,103,0.503660,0.066427,7.582140
2,IGM,104,0.483658,0.081375,5.943549
9,MAGS,89,0.532361,0.092394,5.761831
0,SPY,104,0.283350,0.052503,5.396881
4,SMH,104,0.660100,0.125638,5.253992
1,QQQ,104,0.357346,0.078595,4.546670
5,USD,104,2.170320,0.479452,4.526664
8,FNGS,104,0.575085,0.140676,4.088017
6,TQQQ,104,1.046824,0.362603,2.886972
7,FNGU,104,1.989778,0.808515,2.461029
